### Notebook Objective 

Author: Sofia Ingersoll

Date: 2024-07-01


To provide an update on this project to our team, I am subsetting the data that plotted properly in `identifying_ctracts_oversampling.ipynb` and overlaying them to see their overlap relative to the CalEnviroScreen data. An output of 2 csv files containing census tracts that overlapped, and census tracts that did not overlap will be produced, alongside some engaging visualizations. The objective is to create a framework that will be later leveraged once the Climate Vulnerability Metric and EJScreen data are wrangled.


#### Rough Game Plan

##### Quick Mapping
- Read in simple the CA tract data
- Convert everything into geospatial objects
- Assess and identify components provided in each data set (Get familair with data)
- Match CRS + resolution
- Overlay maps to visualize areas without intersectionality 

##### Surveying Application of Results
- Create a xlsx of the census tracts that have zero intersection
- Create a xlsx of the census tracts with the most overlap

##### Interpretting
- What % of CA tracts are within the overlapping regions?
- What is the approximate population within those regions?
- Same questions for those regions **not** intersecting

In [2]:
# loading libraries
import os
import pandas as pd
import numpy as np
import geopandas as gpd 
import xarray as xr 
import rioxarray as rioxr
import rasterio 
import matplotlib.pyplot as plt

# Function to convert column names to snake_case
def to_snake_case(df):
    df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('\n', '_').str.lower()
    return df

##### Read n' Wrangle
Okay here we are going to read in the data that were successful in the `identifying_ctracts_oversampling.ipynb` and overlay them.

The data that was being difficult in the other notebook will be excluded for now and added once the cleaning is complete. Climate & Vulnerability Mertics are being a pain bc the files lack complete information regarding geolocation and/or fsip (tract id). EJScreen is also being tempermental.

For consistency's sake CRS EPSG:2227 - NAD83 / California zone 3 (ftUS) was selected because it is best suited for the state of California.

**CalEnviroScreen data** 

Key attributes we'll want to look at easily are `tract`,

In [3]:
# import CES shapefile
calenviro = gpd.read_file('data/CalEnviroScreen/CES4 Final Shapefile.shp')

# clean naming for easier 
calenviro = to_snake_case(calenviro)

# crs config
calenviro = calenviro.to_crs('EPSG:2227')

calenviro

,tract,zip,county,approxloc,totpop19,ciscore,ciscorep,ozone,ozonep,pm2_5,...,elderly65,hispanic,white,africanam,nativeam,othermult,shape_leng,shape_area,aapi,geometry
0,6.083002e+09,93454,Santa Barbara,Santa Maria,4495,36.019653,69.162885,0.034190,10.566273,7.567724,...,12.5028,68.9210,20.8899,0.4004,0.2670,1.3126,6999.357689,2.847611e+06,8.2091,"POLYGON ((6580888.757 1071692.998, 6586374.203..."
1,6.083002e+09,93455,Santa Barbara,Santa Maria,13173,37.030667,70.637922,0.035217,11.561917,7.624775,...,5.3519,78.6229,13.2240,2.5051,0.0000,0.9489,19100.578232,1.635292e+07,4.6990,"POLYGON ((6580888.757 1071692.998, 6580894.664..."
2,6.083002e+09,93454,Santa Barbara,Santa Maria,2398,31.213140,61.069087,0.034190,10.566273,7.548835,...,12.8857,65.7214,30.6088,0.9591,0.0000,2.1685,4970.985897,1.352329e+06,0.5421,"POLYGON ((6586394.940 1074311.362, 6586374.203..."
3,6.083002e+09,93455,Santa Barbara,Orcutt,4496,6.639331,5.988401,0.036244,13.615432,7.660570,...,14.4128,22.9537,69.1948,0.9342,0.7117,2.5356,6558.956012,2.417717e+06,3.6699,"POLYGON ((6589067.886 1050036.969, 6589362.670..."
4,6.083002e+09,93455,Santa Barbara,Orcutt,4008,14.022852,23.121533,0.036244,13.615432,7.663210,...,18.8872,33.4082,59.7804,0.6986,1.4721,1.3723,6570.368730,2.608422e+06,3.2685,"POLYGON ((6582087.580 1050102.451, 6586100.333..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8030,6.037430e+09,91016,Los Angeles,Monrovia,5339,17.124832,30.610187,0.062365,88.699440,11.873339,...,17.4752,28.7132,53.3995,1.5733,0.0000,7.1549,7166.130635,1.938016e+06,9.1590,"POLYGON ((7321847.795 796834.152, 7322136.180 ..."
8031,6.037431e+09,91007,Los Angeles,Arcadia,4365,13.841990,22.566818,0.059387,79.987554,11.816074,...,10.4926,10.9507,26.3918,3.3677,0.0000,3.3677,3941.781806,4.855630e+05,55.9221,"POLYGON ((7304547.010 787317.104, 7304590.963 ..."
8032,6.037431e+09,91016,Los Angeles,Monrovia,6758,39.697849,74.508321,0.061338,84.579963,11.892654,...,7.2951,58.2273,16.1438,8.9967,0.0000,1.1098,8020.091253,3.015661e+06,15.5223,"POLYGON ((7319823.634 792917.766, 7319841.586 ..."
8033,6.037534e+09,90201,Los Angeles,Bell,6986,62.931044,97.049924,0.046325,46.994400,12.019728,...,9.4188,91.4114,6.9425,0.6728,0.2577,0.7157,4949.116808,8.118955e+05,0.0000,"POLYGON ((7264190.561 728923.484, 7264048.647 ..."


**SB535 Disadvantaged Communities data**

Key attributes we'll want to look at easily are `census_tract`,

In [4]:
# import SB535 Disadvantaged Communities data
sb535_filename = 'data/SB535_disadv_communities/SB535DACresultsdatadictionary_F_2022.xlsx'

sb535 = pd.read_excel(sb535_filename, sheet_name=None)

# collect sb535 sheet names
sb535_sheetnames = list(sb535.keys())

# read in sheets as df
sb535_1 = sb535[sb535_sheetnames[1]]

sb535_2 = sb535[sb535_sheetnames[2]]

sb535_3 = sb535[sb535_sheetnames[3]]

# combine into df
sb535_df = to_snake_case(pd.concat([sb535_1, sb535_2, sb535_3], ignore_index=True))

# create geo object for mapping
sb535_geo = gpd.GeoDataFrame(sb535_df,
                             geometry = gpd.points_from_xy(sb535_df.longitude, sb535_df.latitude),
                             crs = calenviro.crs)

sb535_geo

,census_tract,calenviroscreen_4.0_score,calenviroscreen_4.0_percentile,calenviroscreen_4.0__percentile_range,pollution_burden_percentile,population_characteristics_percentile,total_population,california_county,approximate_zip_code,approximate_location,...,poverty,poverty_pctl,unemployment,unemployment_pctl,housing_burden,housing_burden_pctl,pop._char.,pop._char._score,pop._char._pctl,geometry
0,6.001401e+09,40.706036,76.147252,75-80%,66.409459,74.848714,2471.0,Alameda,94608.0,Oakland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT EMPTY
1,6.001401e+09,43.736745,80.408472,80-85%,81.829496,70.272315,6133.0,Alameda,94608.0,Oakland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT EMPTY
2,6.001401e+09,43.677589,80.345436,80-85%,73.976353,76.059002,3921.0,Alameda,94609.0,Oakland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT EMPTY
3,6.001401e+09,45.084138,82.123046,80-85%,83.385190,71.583459,4732.0,Alameda,94608.0,Oakland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT EMPTY
4,6.001402e+09,44.528208,81.429652,80-85%,78.780336,74.394856,2839.0,Alameda,94608.0,Oakland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4763,6.113010e+09,NaN,NaN,NaN,NaN,NaN,5355.0,Yolo,NaN,West Sacramento,...,65.9,95.502513,13.3,94.833420,23.7,76.349810,69.562620,7.215905,79.538578,POINT (-121.564 38.564)
4764,6.113010e+09,NaN,NaN,NaN,NaN,NaN,5189.0,Yolo,NaN,West Sacramento,...,51.2,83.241206,11.1,89.927121,24.2,77.820025,68.420473,7.097427,77.975290,POINT (-121.532 38.578)
4765,6.115040e+09,NaN,NaN,NaN,NaN,NaN,4770.0,Yuba,NaN,Marysville,...,52.0,84.208543,7.3,69.976575,21.6,69.150824,63.864399,6.624814,71.293495,POINT (-121.594 39.152)
4766,6.115040e+09,NaN,NaN,NaN,NaN,NaN,2396.0,Yuba,NaN,Linda,...,77.5,99.095477,12.7,93.714211,16.7,47.617237,68.496114,7.105273,78.088754,POINT (-121.571 39.128)


**SD Climate Equity Index (SD_CEI) data**

Key attributes we'll want to look at easily are `tract`, `cei_score` and `cei_class`

In [5]:
# import sd climate equity index data
sd_cei = gpd.read_file('data/sd_climate_equity_index/City_of_San_Diego_CEI_2021_Revision.shp')

sd_cei = to_snake_case(sd_cei)

# match crs
sd_cei = sd_cei.to_crs(calenviro.crs)

sd_cei

,tract,cei_score_,cei_class_,cei_score1,cei_class1,flood_risk,fire_risk,tree_cover,urban_heat,proximity_,...,bikeabilit,access_to_,traffic_de,electric_v,asthma_rat,cancer_fat,healthy_fo,low_infant,cardiovasc,geometry
0,6.073000e+09,74,High,77,High,0.00,77.64,24.46,3566.57,5.73,...,5.80,0.30,942.39,0.0,33.52,23.26,47.30,1.79,7.23,"POLYGON ((7586946.163 291645.762, 7586947.459 ..."
1,6.073000e+09,73,High,79,High,0.00,59.54,24.81,4141.78,5.18,...,10.70,0.25,1011.62,3.0,33.50,23.21,12.97,1.81,8.58,"POLYGON ((7590659.401 291140.646, 7590674.526 ..."
2,6.073000e+09,78,High,83,Very High,0.00,73.55,19.44,4152.74,4.64,...,14.06,0.29,1576.93,18.0,33.75,23.09,12.40,1.79,6.42,"POLYGON ((7593491.575 291712.769, 7593399.774 ..."
3,6.073000e+09,79,High,84,Very High,0.00,86.33,23.17,4880.84,4.75,...,40.09,0.37,2666.02,0.0,36.44,24.79,4.44,1.79,5.45,"POLYGON ((7597036.389 296827.130, 7596961.942 ..."
4,6.073001e+09,75,High,80,High,0.00,59.99,17.04,4852.81,4.13,...,52.69,0.32,1235.97,3.0,33.63,23.28,0.83,1.81,8.09,"POLYGON ((7594235.743 293568.604, 7594335.187 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,6.073004e+09,18,Very Low,32,Low,14.61,21.38,11.22,4353.84,5.89,...,28.66,0.25,700.83,0.0,63.15,20.76,28.72,1.77,7.89,"POLYGON ((7604744.674 278298.173, 7604761.978 ..."
293,6.073005e+09,17,Very Low,19,Very Low,0.00,0.02,9.56,3156.03,5.12,...,29.08,0.25,1565.15,0.0,85.50,23.03,0.00,1.85,9.63,"POLYGON ((7600771.825 275831.530, 7600774.090 ..."
294,6.073005e+09,0,Very Low,0,Very Low,30.66,4.46,4.35,2652.54,2.87,...,12.15,0.30,1144.32,1.0,85.50,23.02,0.00,1.79,9.63,"POLYGON ((7599383.719 273344.915, 7599387.540 ..."
295,6.073010e+09,13,Very Low,29,Low,32.06,93.84,19.37,1179.81,2.37,...,7.16,0.39,45687.87,0.0,37.54,25.26,56.84,1.55,6.02,"POLYGON ((7621642.896 225151.420, 7622213.439 ..."


**White House Map data**

Key attributes we'll want to look at is `geoid10`, it may require wrangling to match other tract formats



In [6]:
# import white house map data
wh_map = gpd.read_file('data/white_house_map/usa/usa.shp',
                       crs = calenviro.crs)

wh_map = to_snake_case(wh_map)

# subset for CA
wh_ca = wh_map[wh_map.sf == 'California']

wh_ca

,geoid10,sf,cf,df_pfs,af_pfs,hdf_pfs,dsf_pfs,ebf_pfs,ealr_pfs,eblr_pfs,...,age_10,age_middle,age_old,ta_cou_116,ta_count_c,ta_perc,ta_perc_fe,ui_exp,thrhld,geometry
3552,06085507904,California,Santa Clara County,0.20,0.00,0.08,0.36,0.02,NaN,0.01,...,0.09,0.73,0.16,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-121.99752 37.30967, -121.99741 37.3..."
3553,06085508504,California,Santa Clara County,0.04,0.00,0.02,0.54,0.00,NaN,0.02,...,0.16,0.75,0.07,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-122.01407 37.35704, -122.01407 37.3..."
3554,06085508505,California,Santa Clara County,0.12,0.00,0.07,0.67,0.00,NaN,0.01,...,0.13,0.68,0.18,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-122.01407 37.35687, -122.01407 37.3..."
3555,06085508704,California,Santa Clara County,0.02,0.00,0.01,0.66,0.00,NaN,0.01,...,0.12,0.82,0.05,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-121.99618 37.38845, -121.99598 37.3..."
3556,06085509403,California,Santa Clara County,0.04,0.05,0.01,0.35,0.00,NaN,0.01,...,0.14,0.82,0.03,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-122.09960 37.39952, -122.09958 37.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11604,06037104203,California,Los Angeles County,0.75,0.24,0.33,0.59,0.59,NaN,0.05,...,0.14,0.78,0.07,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-118.41730 34.27899, -118.41799 34.2..."
11605,06037104204,California,Los Angeles County,0.75,0.18,0.37,0.39,0.57,0.18,0.23,...,0.08,0.80,0.10,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-118.39399 34.28675, -118.39521 34.2..."
11606,06037104403,California,Los Angeles County,0.65,0.05,0.21,0.65,0.68,NaN,0.05,...,0.11,0.75,0.13,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-118.43606 34.26250, -118.43527 34.2..."
11607,06037104404,California,Los Angeles County,0.74,0.16,0.29,0.64,0.61,NaN,0.05,...,0.17,0.74,0.08,NaN,None,NaN,NaN,Nation,21,"POLYGON ((-118.42298 34.26545, -118.42374 34.2..."


**DOE's Energy Justice Mapping Tool - Disadvantaged Communities Reporter (EJMT) data**

Key attributes we'll want to look at is `geoid`, it may require wrangling to match other tract formats

In [7]:
# import doe energy justice mapping tool data
doe_ejmt = gpd.read_file('data/DOE_EJ_mapping_tool/DAC Shapefiles (v2022c).shp',
                         crs = calenviro.crs)

doe_ejmt = to_snake_case(doe_ejmt)

doe_ejmt_ca = doe_ejmt[doe_ejmt.stateabb == 'CA']

doe_ejmt_ca

,geoid,city,county,stateabb,countyfips,cbsa,cbsaname,statefips,population,dacsts,...,disablpct,incplmbpct,snglpntpct,mobilhmpct,nongridpct,lesshspct,lowincfpct,lowincapct,eal_npctl,geometry
263,06003010000,Markleeville,"Alpine County, CA",CA,6003,99999,None,6,1039.0,0,...,0.159769,0.000000,0.386503,0.017946,0.848571,0.088312,0.388835,0.232877,98.801193,"POLYGON ((-13366398.426 4679189.192, -13365864..."
264,06007001300,Chico,"Butte County, CA",CA,6007,17020,"Chico, CA",6,3689.0,0,...,0.173486,0.000000,0.485294,0.034864,0.032665,0.200159,0.514184,0.539340,91.930051,"POLYGON ((-13562009.036 4826320.661, -13561775..."
266,06015000203,Klamath,"Del Norte County, CA",CA,6015,18860,"Crescent City, CA",6,1251.0,0,...,0.301920,0.008811,0.634615,0.432810,0.607930,0.181319,0.539267,0.416000,75.575688,"POLYGON ((-13820782.324 5120426.256, -13819546..."
267,06023010102,Willow Creek,"Humboldt County, CA",CA,6023,21700,"Eureka-Arcata, CA",6,2423.0,0,...,0.210196,0.012333,0.100683,0.073875,0.770812,0.064706,0.385290,0.447674,84.305531,"POLYGON ((-13807602.096 5081143.191, -13803750..."
268,06023940000,None,"Humboldt County, CA",CA,6023,21700,"Eureka-Arcata, CA",6,3263.0,0,...,0.205648,0.062147,0.740831,0.334879,0.841808,0.120198,0.651013,0.383959,70.934437,"POLYGON ((-13782844.641 5035659.850, -13773282..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71260,06071010417,Morongo Valley,"San Bernardino County, CA",CA,6071,40140,"Riverside-San Bernardino-Ontario, CA",6,2931.0,0,...,0.176731,0.000000,0.309859,0.075137,0.258493,0.076227,0.397134,0.277683,89.126878,"POLYGON ((-12982335.051 4038245.509, -12982079..."
71262,06071010415,Twentynine Palms,"San Bernardino County, CA",CA,6071,40140,"Riverside-San Bernardino-Ontario, CA",6,5316.0,0,...,0.232675,0.000000,0.465098,0.015801,0.048458,0.147978,0.297216,0.283658,88.066924,"POLYGON ((-12924994.381 4051867.053, -12924972..."
71264,06071010421,Twentynine Palms,"San Bernardino County, CA",CA,6071,40140,"Riverside-San Bernardino-Ontario, CA",6,6003.0,0,...,0.200203,0.012821,0.434954,0.017824,0.074204,0.084351,0.630327,0.417069,90.001237,"POLYGON ((-12921899.700 4045531.610, -12921576..."
72940,06059990100,None,"Orange County, CA",CA,6059,31080,"Los Angeles-Long Beach-Anaheim, CA",6,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,None


### Overlaying

I want to reduce the size of the data as much as possible before performing inner-join and outer-joins on the the data. So, we're going to only select the most pertenent information from each dataset to join with using SQL + duckdb

In [ ]:
-- SQL to combine data
